In [1]:
import pandas as pd
import geopandas as gpd 
import numpy as np 
import math
#import matplotlib.pyplot as plt
#import palettable as pltt
#from seaborn import palplot
#import contextily as cx
import osmnx as ox

/opt/anaconda3/envs/Thesis2/lib/python3.8/site-packages/geopandas/_compat.py:111: UserWarning: The Shapely GEOS version (3.10.2-CAPI-1.16.0) is incompatible with the GEOS version PyGEOS was compiled with (3.10.1-CAPI-1.16.0). Conversions between both will be slow.
  warnings.warn(


In [46]:
#car
path = '/Volumes/Almadisk1TB/Thesis/structured_dir/src/otp/access/'

jobs_cars_B =  pd.read_csv(path + 'car_cycle_walk/traveltimes/traveltimes_health_car_B.csv')
jobs_destinations = pd.read_csv(path + 'pt/destinations/health_destinations.csv')
#origins_B = pd.read_csv(path + 'pt/origins/origins_B.csv')

In [47]:
place_names_A = ['Eden Prairie, Minnesota, USA', 
               'Chaska, Minnesota, USA', 
               'Chanhassen, Minnesota, USA', 
               'Carver, Carver, Minnesota, USA', 
               'Victoria, Minnesota, USA']

place_names_B = ['Apple Valley, Minnesota, USA', 
               'Eagan, Minnesota, USA', 
               'Burnsville, Minnesota, USA', 
               'Rosemount, Minnesota, USA', 
               'Savage, Minnesota, USA']

service_area_B = ox.geocode_to_gdf(place_names_B)
service_area_B = ox.project_gdf(service_area_B)
service_area_B = service_area_B.to_crs(epsg=4326)

/opt/anaconda3/envs/Thesis2/lib/python3.8/site-packages/osmnx/geocoder.py:110: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  gdf = gdf.append(_geocode_query_to_gdf(q, wr, by_osmid))
/opt/anaconda3/envs/Thesis2/lib/python3.8/site-packages/osmnx/geocoder.py:110: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  gdf = gdf.append(_geocode_query_to_gdf(q, wr, by_osmid))
/opt/anaconda3/envs/Thesis2/lib/python3.8/site-packages/osmnx/geocoder.py:110: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  gdf = gdf.append(_geocode_query_to_gdf(q, wr, by_osmid))
/opt/anaconda3/envs/Thesis2/lib/python3.8/site-packages/osmnx/geocoder.py:110: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Us

In [48]:
jobs_destinations = gpd.GeoDataFrame(
    jobs_destinations, geometry=gpd.points_from_xy(jobs_destinations['LONGITUDE'], jobs_destinations['LATITUDE']))
jobs_destinations = jobs_destinations.set_crs('epsg:4326')

jobs_destinations_within = gpd.sjoin(jobs_destinations, service_area_B, op='within')
tracts = pd.DataFrame(jobs_destinations_within[['GEOID']].value_counts()).reset_index()


/opt/anaconda3/envs/Thesis2/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3301: FutureWarning: The `op` parameter is deprecated and will be removed in a future release. Please use the `predicate` parameter instead.
  if await self.run_code(code, result, async_=asy):


In [49]:
jobs_mt_B

,GEO_ID_origin,GEO_ID_destination,Number of healthcare points,Walk: Walking Distance (meters),Walk: Total Travel Time (seconds),GEOID,0,Microtransit: Total Travel Time (seconds),"Microtransit: Total Travel Time (seconds, including wait time)",Number of shops
0,270190911007006,270190907041,1,124.0,1335,270190907041,1,1542,2442,1
1,270530260281007,270190907041,1,107.0,1197,270190907041,1,1389,2289,1
2,270190908021012,270190907041,1,127.0,1287,270190907041,1,1474,2374,1
3,270190908021011,270190907041,1,155.0,1319,270190907041,1,1472,2372,1
4,270190904014005,270190907041,1,148.0,1511,270190907041,1,1733,2633,1
...,...,...,...,...,...,...,...,...,...,...
5303,270530260261006,270530260193,1,246.0,987,270530260193,1,899,1799,1
5304,270190904012022,270530260193,1,634.0,2641,270530260193,1,2444,3344,1
5305,270530260153014,270530260193,1,258.0,680,270530260193,1,481,1381,1
5306,270190907023009,270530260193,1,237.0,1158,270530260193,1,1135,2035,1


In [50]:
jobs_mt_B = pd.merge(jobs_cars_B, tracts, left_on=['GEO_ID_destination'], right_on=['GEOID'], how='right')
jobs_mt_B['Microtransit: Total Travel Time (seconds)'] = 1.3 * (jobs_mt_B['Walk: Total Travel Time (seconds)'] - (jobs_mt_B['Walk: Walking Distance (meters)'] / 0.83333) )
jobs_mt_B['Microtransit: Total Travel Time (seconds, including wait time)'] = jobs_mt_B['Microtransit: Total Travel Time (seconds)'] + 900
jobs_mt_B[jobs_mt_B < 0] = 0

In [51]:
jobs_mt_B['Number of shops'] = jobs_mt_B['Number of healthcare points'].fillna(0).astype(int)
jobs_mt_B['Microtransit: Total Travel Time (seconds)'] = jobs_mt_B['Microtransit: Total Travel Time (seconds)'].fillna(0).astype(int)
jobs_mt_B['Microtransit: Total Travel Time (seconds, including wait time)'] = jobs_mt_B['Microtransit: Total Travel Time (seconds, including wait time)'].fillna(0).astype(int)
jobs_mt_B

,GEO_ID_origin,GEO_ID_destination,Number of healthcare points,Walk: Walking Distance (meters),Walk: Total Travel Time (seconds),GEOID,0,Microtransit: Total Travel Time (seconds),"Microtransit: Total Travel Time (seconds, including wait time)",Number of shops
0,NaN,NaN,NaN,NaN,NaN,270370607091,1,0,0,0
1,2.703706e+14,2.703706e+11,3.0,246.0,1524.0,270370607273,1,1597,2497,3
2,2.703706e+14,2.703706e+11,3.0,220.0,1408.0,270370607273,1,1487,2387,3
3,2.703706e+14,2.703706e+11,3.0,197.0,1530.0,270370607273,1,1681,2581,3
4,2.703706e+14,2.703706e+11,3.0,237.0,1796.0,270370607273,1,1965,2865,3
...,...,...,...,...,...,...,...,...,...,...
9891,2.703706e+14,2.703706e+11,1.0,217.0,1736.0,270370610073,1,1918,2818,1
9892,2.703706e+14,2.703706e+11,1.0,370.0,2045.0,270370610073,1,2081,2981,1
9893,2.703706e+14,2.703706e+11,1.0,200.0,1369.0,270370610073,1,1467,2367,1
9894,2.703706e+14,2.703706e+11,1.0,193.0,751.0,270370610073,1,675,1575,1


In [52]:
jobs_mt_B.to_csv(path + 'car_cycle_walk/traveltimes/traveltimes_health_mt_B.csv')